# InViz: Live Computation Example

In this notebook, we'll see how you can plug any function you want into InViz to help visualize the relationship between parameters and observables.

In [ ]:
import inviz as nv
import holoviews as hv
from holoviews import opts
import numpy as np
import pandas as pd
from scipy import signal

Suppose we want to visualize how changes in amplitude, frequency, and phase affect some wave functions. Let's choose the sine and sinc functions, and we'll also throw in a sawtooth wave just for fun.

A sine wave can be defined as:
$$
y(x) = A \sin{(2\pi f x + \phi)} = A \sin{(\omega x + \phi)}
$$
where $A$ is the amplitude, $f$ is the frequency, $\omega$ is the angular frequency, and $\phi$ is the phase.

The sinc function is simply:
$$
\mathrm{sinc}~(x) = \frac{\sin{(x)}}{x}
$$

The sawtooth wave is a bit more complicated. You can read more about it [here](https://en.wikipedia.org/wiki/Sawtooth_wave).

First let's generate some random data:

In [ ]:
# Set up the parameters of the problem.
ndim, nsamples = 3, 1000

# Generate some fake data.
np.random.seed(42)
data1 = np.random.randn(ndim * 4 * nsamples // 5).reshape(
    [4 * nsamples // 5, ndim]
)
data2 = 4 * np.random.rand(ndim)[None, :] + np.random.randn(
    ndim * nsamples // 5
).reshape([nsamples // 5, ndim])
data = np.vstack([data1, data2])

Now let's convert it into a pandas DataFrame (InViz only accepts DataFrames, for now) and add labels with some fancy $\LaTeX$ formatting:

In [ ]:
param_names = ['frequency', 'phase', 'amplitude']
latex = ['\omega / 2\pi', '\phi', '\mathrm{amplitude}']
df = pd.DataFrame(data, columns=param_names)
latex_dict = dict(zip(params, latex))

Next, we will define the function that takes a row of input data and uses it to compute the three waveforms. Note that the function must return data in a dictionary with the following format:

```python
results = {
    'observable 1': {'x': x, 'a': a},
    'observable 2': {'y': y, 'b': b},
    'observable 3': {'z': z, 'c': c}, 
}
```

The values x, y, z will go on the horizontal axes of each graph, and a, b, c will go on the vertical axes. The keys will be used as the title and axis labels of each respective plot. 

Like so:

In [ ]:
def example_observables(index, input_data):
    selection = input_data.iloc[[index]]
    x = np.linspace(-2*np.pi, 2*np.pi, 201)
    angular_freq = 2*np.pi*float(selection['frequency'])
    phase = float(selection['phase'])
    amp = float(selection['amplitude'])
    sin = amp * np.sin(angular_freq*x + phase)
    sinc = amp * np.sinc(angular_freq*x/np.pi + phase)
    sawtooth = amp * signal.sawtooth(angular_freq * x + phase)
    funcs = {
        'Sine': {'x': x, 'sin(x)': sin}, 
        'Sinc': {'x': x, 'sinc(x)': sinc}, 
        'Sawtooth': {'x': x, 'sawtooth(x)': sawtooth}}
    return funcs
    

Then, we use a HoloViews `opts` object to customize the observable plots.

In [ ]:
exopts = opts.Curve(xlim=(-2*np.pi, 2*np.pi), height=400, width=500, color=hv.Cycle('GnBu'), bgcolor='#22262F')

Finally, we will use `nv.viz` to interactively visualize the whole thing. Here's an overview of the function arguments:
- __data__: (Pandas DataFrame) the data you want shown as a scatter plot
- __data_observable__: (Pandas DataFrame) the observables you'd like to plot
- __myfunction__: (function) your function that computes some observables based on a data point
- __myfunction_args__: (tuple) the arguments of that function
- __show_observables__: (Boolean) whether you want to see the observable plots or not 
- __latex_dict__: (dict) a dictionary containing the Latex formatting for your axis labels (default = None)
- __curve_opts__: (HoloViews Options) the option builder used by HoloViews to customize plots ([more info](https://holoviews.org/user_guide/Applying_Customizations.html#option-builders))

**IMPORTANT**: only pass arguments into __either__ `data_observable` __or__ `myfunction + myfunction_args`, depending on if you have a precalculated observable dataset, or you want to dynamically calculate them with a custom function.

Run the cell below to test out the interactivity by selecting points on the scatterplot in the top section, and see what appears on the plots in the bottom section!

In [ ]:
nv.viz(data=df, myfunction=example_observables, myfunction_args=(df,), show_observables=True, latex_dict=latex_dict, curve_opts=exopts)